## 9. Containers and Microservices

Two important things of Linux OS:

1. One is the file system hierarchy of an OS

![](https://media.licdn.com/dms/image/D5612AQGuHl_yMRdiQw/article-cover_image-shrink_600_2000/0/1680372550971?e=2147483647&v=beta&t=Dm2fjW6vnaTNiaIkYFOCXq5O3-L9xhvHyhVNYQQ0Cj0)

2. The processes (from services like Tomcat, Nginx or MongoDB) that run the OS file system, which are part of a process tree

![](https://media.cheggcdn.com/media/a1a/a1a2b753-0c70-43b0-9132-d31296c65f62/php78vpA2)

When multiple processes (services) are running in the same OS file system, changes or modifications made to it by a process may affect other processes: 

* That's why we need to **Isolate** these processes in different computers/servers. So, each process has its own OS, files and process tree and not disturbed by other process.

* This creates a problem: More computers = More money.

* This is where **Containers** come in: a container is just a directory which contains its own file system, similar to the OS file system (like a mini OS), and can be all connected together through a virtualised network.

    - For example, a container for Nginx process would only contain directories relevant to the process where the pid = 1 would be the main process (i.e., nginx (pid=1))

    - Each container is connected through an assigned IP address.

    - Each container is then of very light weight -> Can be archived as an image/container image.

    - The purpose of archiving in container images is to ship these anywhere you want.

This containarisation is performed on top of the host OS of a VM or computer:

![](https://www.netscaler.com/content/dam/netscaler/images/graphics/infographics/what-is-containerization.png)

* The most famous container runtime environment is *docker*.

### Docker

Docker documentation: https://docs.docker.com/get-started/

The process to set up docker:

* Create a VM

* Install docker engine (docker daemon)

* Run the containers using the images from a registry

* Website for registry images search: https://hub.docker.com/

* You can select any official image or create your own one and store it on Docker Hub.

![](https://docs.docker.com/assets/images/architecture.svg)

### Docker Commands

* Setup (provisioning):

    ```
    config.vm.provision "shell", inline: <<-SHELL
        
        # Ubuntu update and basic dependencies
        sudo apt-get update
        sudo apt-get install \
            ca-certificates \
            curl \
            gnupg -y

        # Download and install docker
        sudo install -m 0755 -d /etc/apt/keyrings
        curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
        sudo chmod a+r /etc/apt/keyrings/docker.gpg

        echo \
            "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
            "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
            sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

        sudo apt-get update
        sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin -y
        SHELL
    end
    ```

* Check system status and run an image (`hello-world`):

    ```
    $ vagrant ssh
    $ sudo -i
    # systemctl status docker

    # docker run hello-world
    ```

* Check available images on your system:

    ```
    # docker images
    ```

* Check all the containers:

    ```
    # docker ps -a
    ```

* Run a docker container with a given name and host port:

    ```
    # docker run --name web01 -d -p 9080:80 nginx
    ```

    where `--name web01` means that the container will be named web01, `-d` will run it in the background, `9080` is the container port, `80` is the host (VM) port and `nginx` is the image.

To access the container from outside, you need to do **port mapping**: access host IP with the port 9080, so it routes the request to the container web01 on port 80.

* Check the container id, port and name running `docker ps`:

| CONTAINER ID | IMAGE  | ... | PORTS                                 | NAMES |
|--------------|--------|-----|---------------------------------------|-------|
| 03e1897c1006 | nginx  | ... | 0.0.0.0:9080->80/tcp, :::9080->80/tcp | web01 |

* Find the IP address of the container:

    ```
    # docker inspect web01 # or

    # docker inspect 03e1897c1006 # using container id
    ```

You will find it in "Networks" > "bridge" > "IPAddress": "172.17.0.2"

* We can access it locally now by giving its IP address and the host port:

    ```
    # curl http://172.17.0.2:80
    ```

* Now, from the outside, we can get the VM IP address using `ip addr show` and giving the container port. Go to a browser and give the IP+port `192.168.56.82:9080`


**A glimpse on how to build our own image**:

* Create a `Dockerfile`:

    ```
    # mkdir images

    # cd images/

    # vim Dockerfile
    ```

* Paste the content (webpage application):

    ```
    FROM ubuntu:latest AS BUILD_IMAGE
    RUN apt update && apt install wget unzip -y
    RUN wget https://www.tooplate.com/zip-templates/2128_tween_agency.zip
    RUN unzip 2128_tween_agency.zip && cd 2128_tween_agency && tar -czf tween.tgz * && mv tween.tgz /root/tween.tgz

    FROM ubuntu:latest
    LABEL "project"="Marketing"
    ENV DEBIAN_FRONTEND=noninteractive

    RUN apt update && apt install apache2 git wget -y
    COPY --from=BUILD_IMAGE /root/tween.tgz /var/www/html/
    RUN cd /var/www/html/ && tar xzf tween.tgz
    CMD ["/usr/sbin/apache2ctl", "-D", "FOREGROUND"]
    VOLUME /var/log/apache2
    WORKDIR /var/www/html/
    EXPOSE 80
    ```

    Docker images are always built on top of a current image (official or not): `FROM ubuntu:latest`

* Run `docker build -t <imagename> .`, where `.` is the location of the Dockerfile (current directory)

* Check if the image is in `docker images`

* Run `docker run -d -P tesimg` (`-P` for automatic port number)

* Check if the image is running using `docker ps`. Save the port number for `tesimg`

* Check if the webpage is running on your browser. Get the IP address `ip addr show` and go to the browser and put `ipaddress:portnumber`: `192.168.56.82:32768`

**Stop and remove containers/images**:

* `docker stop <contname1> <contname2>`

* `docker rm <contname1> <contname2>`

* `docker rmi <imgid1> <imgid2>`

### Creating Multiple Containers

Use `docker compose` for the VProfile project:

* This command will read a file called `docker-compose.yaml`. For the course, the file is in https://github.com/hkhcoder/vprofile-project/tree/docker

    - The file contains all the images for the different services to be containeraised: VProfile web for Nginx, VProfile app for Tomcat, Memcached, RabbitMQ and VProfile for database.

    - Click on `raw` and copy the link.

    - In your VM, create a directory and run `wget <link>` inside that directory
    
* If you just run `docker compose`, it will show you the different options.

* Run `docker compose up -d` to bring up all the containers in the background (`-d`) 

* To stop the containers, use `docker compose down`

* To remove all the stopped containers, use `docker system prune -a`

### Containerisation and Microservices

**Monolith vs Microservices**:

Microservices make developing, debugging, mantenance and keeping updates of subservices a lot easier and faster, since they are developed as a whole application (with their own programming language or technology) and isolated from other services (independent, self-contained).

![](https://miro.medium.com/max/6996/1*xu1Ge_Cew0DHdSU6ETcpLQ.png)

**Containerisation and Microservices**:

Containerisation is KEY in the software archictectures based on microservices.

![](https://images.prismic.io/qovery/d84bc61f-b2c6-40cf-b045-3970a9d15065_Untitled.png?auto=compress,format)

### Emart Application

A microservice-architecture application:

* API Gateway (Nginx): Three end points: root (`/`), `/api` and `/webapi`. 

* When the user access the url of the website, it will be redirected to root -> Angular app (frontend or client app)

* And this will in turn connect to `/api` -> NodeJS app + Mongo DB (Emart application)

* Another feature is `/webapi` -> Java application + MySQL DB (Books api)

* From a developers point of view, the microservices architecture is very scalable since you can start like this and add other microservices.

![](https://miro.medium.com/v2/resize:fit:1200/0*t72lUE5lC4hOjYoa.jpg)

The source code for this application is in https://github.com/devopshydclub/emartapp

#### Commands for Emart App

Install the Vagrant VM using the Vagrantfile with the provisioning:

```
sudo apt-get update
sudo apt-get install \
ca-certificates \
curl \
gnupg -y

sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg
echo \
"deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
"$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin -y
```

Inside the VM:

* Go to root user: `sudo -i`

* Go to the repository and copy the html link. Then, run the command

`git clone https://github.com/devopshydclub/emartapp.git`

* You will find the emartapp folder by running `ls`. Run `cd emartapp` to go inside the folder

* Open `docker-compose.yaml` file:

    - You will see a step called **build** which is used to build images from Docker files.

    - You can also pass the path from the Docker repository to build the images

* Check if there are any existing images or containers:

    - Images: `docker images`

    - Containers: `docker ps -a`

* Inside the emartapp folder:

    - Run `docker compose up -d`

    - If an error appears, you can run first `docker compose build` and then `docker compose up -d`

    - Wait until the docker images build

* When finished building, check the images using `docker compose ps`

* To access the app, you need to find the port for the *ngnix* service:

    - Find the port from the output of `docker compose ps` command

    - Run `ip addr show` to check the servers IP address: Select the 3rd one

* Go to a browser and put the IP Address in the search using the port `80`: 

    ` 192.168.56.82:80`

    You will find the app functioning there!

* To remove and clean, run: `docker compose down` and `docker system prune -a`